In [1]:
import sys
from tensorflow import keras
import os  
import torch
import time
import cv2
import numpy as np
from PIL import Image
import psycopg2

In [2]:
def loadRN50():
    return [
        ## S
        {'c': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/connector.h5'),
        'ms': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/multiscrew.h5'),
        's1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/screw1.h5'),
        's2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/screw2.h5'),
        'sp2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/supporter2.h5'),
        'sp3': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_Single/supporter3.h5')},
        ## M
        {'c1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/connector1.h5'),
        'c2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/connector2.h5'),
        'ms1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/multiscrew1.h5'),
        'ms2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/multiscrew2.h5'),
        's1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/screw1.h5'),
        's2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/screw2.h5'),
        'sp1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/supporter1.h5'),
        'sp2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/supporter2.h5'),
        'sp3': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM_Chassis_MULT/supporter3.h5')},
        ## AS
        {'c1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/connector1.h5'),
        'c2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/connector2.h5'),
        'ms': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/multiscrew.h5'),
        's1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/screw1.h5'),
        's2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/screw2.h5'),
        'sp1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/supporter1.h5'),
        'sp2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/supporter2.h5'),
        'sp3': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/supporter3.h5'),
        'sp4': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_Single/supporter4.h5')},
        ## AM
        {'c1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/connector1.h5'),
        'c2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/connector2.h5'),
        'c3': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/connector3.h5'),
        'ms1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/multiscrew1.h5'),
        'ms2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/multiscrew2.h5'),
        's1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/screw1.h5'),
        's2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/screw2.h5'),
        'sp1': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/supporter1.h5'),
        'sp2': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/supporter2.h5'),
        'sp3': keras.models.load_model('/Users/mh/Downloads/models/ResNet50_revised/TM-A_Chassis_MULT/supporter3.h5')}
    ]

def connectToServer():
    conn = None
    try:
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host="ls-491dfa0f47b6c73fb81aa4b4794c82f223f8cc98.cxfmrdeq6htd.ap-northeast-2.rds.amazonaws.com",
                                database="flask_capstone",
                                user="dbmasteruser",
                                password="g<#wHS>79|E!pGF9JJ7!$a[?&nLA?aP%",
                                port=5432)

        print('PostgreSQL database version:')
        cur = conn.cursor()
        cur.execute('SELECT version()')
        db_version = cur.fetchone()
        cur.close()
        print(db_version)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Exception")
        print(error)
    return conn

In [3]:
def downloadImageFromServerAndGetLocalPath(id):
    ## DB에서 id에 해당하는 튜플을 select
    cur = CONN.cursor()
    cur.execute("select image, type from public.image where id=%s", (id, ))
    tup = cur.fetchone()
    cur.close()

    ## select 된 튜플 이미지(binary)를 numpy.ndarray로 변환 + type 저장
    binImage = tup[0].tobytes()
    npImage = cv2.imdecode(np.frombuffer(binImage, np.uint8), -1)
    datasetType = tup[1]

    ## fetch한 이미지를 local에 파일로 저장
    path = f'{round(time.time() * 1000)}.jpg'
    cv2.imwrite(path, npImage)

    ## 저장된 이미지 경로, 데이터셋 type 리턴
    return path, datasetType

In [4]:
%cd /Users/mh/yolov5/
import detect
#%cd ..

def processImage(path, datasetType):
    ## 이미지 읽기
    image = cv2.imread(path)

    ## 결과 이미지
    result = image.copy()

    ## yolo로 부품 탐지
    yoloResults = detect.run(weights=YOLOv5_MODELS_PATH[datasetType], source=path)

    ## yolo 부품 탐지 결과 순회
    for yoloResult in yoloResults:
        ## 결과 추출 (부품 이름, conf., 좌표값)
        spl = yoloResult[0].split(' ')
        part, conf = spl[0], spl[1]
        x1, y1 = int(yoloResult[1][0]), int(yoloResult[1][1])
        x2, y2 = int(yoloResult[1][2]), int(yoloResult[1][3])

        ## 부품 crop
        croppedImage = image[y1:y2, x1:x2]
        croppedImage = cv2.resize(croppedImage, (224, 224))
        croppedImage = np.expand_dims(croppedImage, axis=0)
        
        ## CNN으로 부품 class 분류
        predRN50 = RN50_MODELS[datasetType][part].predict(croppedImage)

        ## 결과를 원본 이미지에 그리기
        if np.argmax(predRN50[0]) == 0:  # 정상
            result = cv2.rectangle(result, (x1, y1), (x2, y2), (0, 255, 0), 5)
            cv2.putText(result, part + ': Normal', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 255, 0), 5)
        else:  # 불량
            result = cv2.rectangle(result, (x1, y1), (x2, y2), (0, 0, 255), 5)
            cv2.putText(result, part + ': Defect', (x1, y1 - 15), cv2.FONT_HERSHEY_SIMPLEX, 2.0, (0, 0, 255), 5)
    
    ## 저장된 이미지 삭제
    os.remove(path)

    return result

/Users/mh/yolov5


In [5]:
def uploadResultToServer(result, id):
    #### 결과 이미지를 binary로 변환
    _, encoded = cv2.imencode('.jpeg', result)
    binImage = encoded.tobytes()

    ### DB 튜플의 result 컬럼 update
    cur = CONN.cursor()
    cur.execute('''update public.image set result=%s where id=%s''', (binImage + b'1', id, ))
    CONN.commit()
    cur.close()

In [6]:
RN50_MODELS = loadRN50()
CONN = connectToServer()

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 14.7 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 7.3.1 20180712 (Red Hat 7.3.1-12), 64-bit',)


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

YOLOv5_MODELS_PATH = [
    '/Users/mh/Downloads/models/yolo_models/best_s.pt',
    '/Users/mh/Downloads/models/yolo_models/best_m.pt',
    '/Users/mh/Downloads/models/yolo_models/best_as.pt',
    '/Users/mh/Downloads/models/yolo_models/best_am.pt'
]

@app.route('/')
def hello(): # Web 기본 접속 화면 (주소 입력하고 들어가면 나오는 default 화면)
  return "<h1>Welcome to Flask</h1>"

@app.route('/process', methods=['POST'])
def handle_request():
  fail = False
  value = request.json.get('value') # image id (DB)

  path, datasetType = downloadImageFromServerAndGetLocalPath(value)
  result = processImage(path, datasetType)
  uploadResultToServer(result, value)
  
  if fail:
    return jsonify({'result': -1})
      
  return jsonify({'result': value}) # 완료 후 image id 반환
  
if __name__ == '__main__':
  app.run(port='5002')

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5002
Press CTRL+C to quit
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.9 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7034398 parameters, 0 gradients
image 1/1 /Users/mh/yolov5/1685494876490.jpg: 384x640 1 c1, 1 c2, 1 ms1, 1 ms2, 1 s1, 1 s2, 1 sp3, 54.3ms
Speed: 1.1ms pre-process, 54.3ms inference, 6.3ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp17


1/1 [==============================] - 0s 321ms/step


127.0.0.1 - - [31/May/2023 10:01:20] "POST /process HTTP/1.1" 200 -
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.9 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7026307 parameters, 0 gradients
image 1/1 /Users/mh/yolov5/1685494885361.jpg: 384x640 1 c, 1 ms, 1 sp2, 43.2ms
Speed: 0.4ms pre-process, 43.2ms inference, 0.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp18


1/1 [==============================] - 0s 322ms/step


...

127.0.0.1 - - [31/May/2023 10:03:06] "POST /process HTTP/1.1" 200 -
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.9 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7034398 parameters, 0 gradients
image 1/1 /Users/mh/yolov5/1685495004016.jpg: 384x640 1 c1, 1 c2, 1 ms1, 1 ms2, 1 s1, 1 s2, 1 sp3, 45.1ms
Speed: 0.6ms pre-process, 45.1ms inference, 0.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp23


1/1 [==============================] - 0s 49ms/step


127.0.0.1 - - [31/May/2023 10:03:25] "POST /process HTTP/1.1" 200 -
YOLOv5 🚀 v7.0-163-g016e046 Python-3.10.9 torch-2.0.1 CPU

Fusing layers... 
YOLOv5s summary: 157 layers, 7026307 parameters, 0 gradients
image 1/1 /Users/mh/yolov5/1685495012268.jpg: 384x640 1 c, 1 ms, 1 s1, 1 s2, 1 sp2, 1 sp3, 45.7ms
Speed: 0.4ms pre-process, 45.7ms inference, 0.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs/detect/exp24


1/1 [==============================] - 0s 48ms/step


127.0.0.1 - - [31/May/2023 10:03:33] "POST /process HTTP/1.1" 200 -
